### Create AML SDK docker file and build and test image

To run this notebook, start it via a plain Jupyter notebook session on your Ubuntu_Azure_VM:

* login (ssh) into the VM:
```
loginname1@Ubuntu_Azure_VM:/datadrive01/prj/PowerAIWithDocker$ jupyter notebook --notebook-dir=/workspace --ip=0.0.0.0 --port=8888 --no-browser --allow-root
```
  
   
* then go to:    
http://Ubuntu_Azure_VM.eastus2.cloudapp.azure.com:9000/ 
  
  
https://docs.microsoft.com/en-us/azure/machine-learning/service/azure-machine-learning-release-notes  
https://hub.docker.com/r/continuumio/miniconda3/tags  
   
Note: As of 02/13/2019, latest versions are:  
Python 3.7.2 (https://www.python.org/downloads/, https://anaconda.org/anaconda/python)  
miniconda3 4.5.12 (https://hub.docker.com/r/continuumio/miniconda3/tags)  
AML SDK  v.1.0.15 (https://docs.microsoft.com/en-us/python/api/overview/azure/ml/install?view=azure-ml-py)  
Ubuntu 19.04 (https://hub.docker.com/_/ubuntu?tab=tags)  

In [1]:
# Allow multiple displays per cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" 

In [2]:
%load_ext dotenv
# import dotenv
# print (dir(dotenv))

In [3]:
%%writefile .env

# Your docker login and image repository name
docker_login = 'georgedockeraccount'
image_tag = "aml-sdk_docker_image"
image_version = 'sdk.v1.0.81' 

Overwriting .env


In [4]:
%dotenv
docker_file_location = 'docker_build'

import os
docker_file_name = 'dockerfile'+ '_' + os.getenv('image_tag') + '_' + os.getenv('image_version')
conda_dependency_file = 'aml_sdk_conda_dep_file.yml'

In [5]:
local_dir = !pwd
docker_file_dir = os.path.join(*([local_dir.s, docker_file_location]))
docker_file_dir
 
docker_file_path = os.path.join(*([docker_file_dir]+[docker_file_name]))
docker_file_path

conda_dependency_file_path = os.path.join(*([docker_file_dir]+[conda_dependency_file]))
conda_dependency_file_path

#clean previous history (i.e. content of amlsdk/docker)
!mkdir -p {docker_file_dir}
!chmod -R ugo=rwx $docker_file_dir
!rm -rf $docker_file_dir/*
!ls -l $docker_file_dir

'/datadrive01/prj/PowerAIWithDocker/amlsdk/docker_build'

'/datadrive01/prj/PowerAIWithDocker/amlsdk/docker_build/dockerfile_aml-sdk_docker_image_sdk.v1.0.81'

'/datadrive01/prj/PowerAIWithDocker/amlsdk/docker_build/aml_sdk_conda_dep_file.yml'

total 0


In [6]:
%%writefile $conda_dependency_file_path
name: aml_sdk_conda_env02
    
channels:
  - anaconda
dependencies:
  - python=3.7 # 3.6 req by tf, which seems to be req by automl, not 3.7.2 
  - numpy
  - cython
  - notebook 
  - nb_conda
  - scikit-learn
  - pip
  - pip:
    - python-dotenv
    - papermill[azure]
    - azureml-sdk[notebooks,automl,explain] 

Writing /datadrive01/prj/PowerAIWithDocker/amlsdk/docker_build/aml_sdk_conda_dep_file.yml


In [7]:
%%writefile $docker_file_path 

# FROM continuumio/miniconda3:4.6.14 as AzureMLSDKOnly 
FROM continuumio/miniconda3:4.7.12 as AzureMLSDKOnly    
MAINTAINER George Iordanescu <ghiordan@microsoft.com>

RUN apt-get update --fix-missing && apt-get install -y --no-install-recommends \
    gpg-agent \
    gcc g++ \
    wget bzip2 \
    curl apt-transport-https lsb-release gpg \
    git make \
    sudo && \
    apt-get clean && \
    rm -rf /var/lib/apt/lists/*

ADD aml_sdk_conda_dep_file.yml /tmp/aml_sdk_conda_dep_file.yml
ENV CONDA_DIR /opt/conda
ENV CONDA_ENV_NAME aml-sdk-conda-env    

# reate aml sdk conda env defined by .yml file
RUN /opt/conda/bin/conda env create -q --name $CONDA_ENV_NAME -f /tmp/aml_sdk_conda_dep_file.yml && \
    /opt/conda/bin/conda clean  --yes --all 
ENV CONDA_AUTO_UPDATE_CONDA=false
ENV CONDA_DEFAULT_ENV=$CONDA_ENV_NAME
ENV CONDA_PREFIX=$CONDA_DIR/envs/$CONDA_DEFAULT_ENV
ENV PATH=$CONDA_PREFIX/bin:/opt/conda/bin:$PATH                
        
FROM AzureMLSDKOnly as AzureMLSDKAndAzCli
# Install Azure CLI
ENV AZ_CLI_REPO=stretch   
RUN echo "deb [arch=amd64] https://packages.microsoft.com/repos/azure-cli/ $AZ_CLI_REPO main" | \
    tee /etc/apt/sources.list.d/azure-cli.list && \
    curl -L https://packages.microsoft.com/keys/microsoft.asc | apt-key add - && \
    apt-get update && \
    apt-get install -y --no-install-recommends \
    azure-cli 
    
# Install AzCopy
RUN mkdir -p /tmp/azcopy && mkdir -p /azcopy10 && \
    wget -O /tmp/azcopy/azcopyv10.tar.gz https://aka.ms/downloadazcopy-v10-linux &&  \
    tar -xf /tmp/azcopy/azcopyv10.tar.gz -C /azcopy10 
RUN rm -rf /tmp/azcopy
ENV PATH=/azcopy10/azcopy_linux_amd64_10.3.2:$PATH
    
FROM AzureMLSDKAndAzCli as AzureMLSDKAndAzCliAndAZDevOps
#https://marketplace.visualstudio.com/items?itemName=ms-vsts.cli
RUN az extension add --name azure-devops
        
CMD /bin/bash


Writing /datadrive01/prj/PowerAIWithDocker/amlsdk/docker_build/dockerfile_aml-sdk_docker_image_sdk.v1.0.81


In [8]:
%dotenv

#add docker image version info only if image_version has been set to a non-empty string, otherwise let docker increment it
image_version_string = os.getenv('image_version')
if image_version_string!="":
    image_version_string = ':'  + image_version_string
    
aml_docker_image_name = os.getenv('docker_login') + '/' + os.getenv('image_tag') + \
image_version_string
# aml_sudo_docker_image_name = os.getenv('docker_login') + '/' + os.getenv('image_tag') + \
# 'sudo' + image_version_string
aml_azcli_docker_image_name = os.getenv('docker_login') + '/' + os.getenv('image_tag') + \
'azcli' + image_version_string
aml_azcli_azdevops_docker_image_name = os.getenv('docker_login') + '/' + os.getenv('image_tag') + \
'azcliazdevops'  + image_version_string


docker_file_path
aml_docker_image_name
# aml_sudo_docker_image_name
aml_azcli_docker_image_name
aml_azcli_azdevops_docker_image_name

docker_file_dir

working_path = docker_file_dir
! ls -l $working_path


'/datadrive01/prj/PowerAIWithDocker/amlsdk/docker_build/dockerfile_aml-sdk_docker_image_sdk.v1.0.81'

'georgedockeraccount/aml-sdk_docker_image:sdk.v1.0.81'

'georgedockeraccount/aml-sdk_docker_imageazcli:sdk.v1.0.81'

'georgedockeraccount/aml-sdk_docker_imageazcliazdevops:sdk.v1.0.81'

'/datadrive01/prj/PowerAIWithDocker/amlsdk/docker_build'

total 8
-rw-rw-r-- 1 loginvm0_1 loginvm0_1  313 Dec 25 00:04 aml_sdk_conda_dep_file.yml
-rw-rw-r-- 1 loginvm0_1 loginvm0_1 1840 Dec 25 00:04 dockerfile_aml-sdk_docker_image_sdk.v1.0.81


In [9]:
build_command = 'docker build -t ' + aml_docker_image_name  + \
 ' --target AzureMLSDKOnly -f ' + \
docker_file_path +' '+ working_path + ' --no-cache'
build_command

!     $build_command
!     docker build -t $aml_azcli_docker_image_name  --target AzureMLSDKAndAzCli -f $docker_file_path $working_path
!     docker build -t $aml_azcli_azdevops_docker_image_name -f $docker_file_path $working_path


'docker build -t georgedockeraccount/aml-sdk_docker_image:sdk.v1.0.81 --target AzureMLSDKOnly -f /datadrive01/prj/PowerAIWithDocker/amlsdk/docker_build/dockerfile_aml-sdk_docker_image_sdk.v1.0.81 /datadrive01/prj/PowerAIWithDocker/amlsdk/docker_build --no-cache'

Sending build context to Docker daemon  4.608kB
Step 1/11 : FROM continuumio/miniconda3:4.7.12 as AzureMLSDKOnly
4.7.12: Pulling from continuumio/miniconda3

62c62ec6: Pulling fs layer 
c0154f16: Pulling fs layer 
Digest: sha256:6c979670684d970f8ba934bf9b7bf42e77c30a22eb96af1f30a039b484719159
Status: Downloaded newer image for continuumio/miniconda3:4.7.12
 ---> 406f2b43ea59
Step 2/11 : MAINTAINER George Iordanescu <ghiordan@microsoft.com>
 ---> Running in f1aa1cbe557b
Removing intermediate container f1aa1cbe557b
 ---> c8b5e5986ff6
Step 3/11 : RUN apt-get update --fix-missing && apt-get install -y --no-install-recommends     gpg-agent     gcc g++     wget bzip2     curl apt-transport-https lsb-release gpg     git make     sudo &&     apt-get clean &&     rm -rf /var/lib/apt/lists/*
 ---> Running in 4f63fa06364f
Get:1 http://deb.debian.org/debian buster InRelease [122 kB]
Get:2 http://security.debian.org/debian-security buster/updates InRelease [65.4 kB]
Get:3 http://deb.debian.org/debi

debconf: delaying package configuration, since apt-utils is not installed
Fetched 57.9 MB in 1s (107 MB/s)
Selecting previously unselected package libpython3.7-minimal:amd64.
(Reading database ... 12557 files and directories currently installed.)
Preparing to unpack .../libpython3.7-minimal_3.7.3-2_amd64.deb ...
Unpacking libpython3.7-minimal:amd64 (3.7.3-2) ...
Selecting previously unselected package python3.7-minimal.
Preparing to unpack .../python3.7-minimal_3.7.3-2_amd64.deb ...
Unpacking python3.7-minimal (3.7.3-2) ...
Setting up libpython3.7-minimal:amd64 (3.7.3-2) ...
Setting up python3.7-minimal (3.7.3-2) ...
Selecting previously unselected package python3-minimal.
(Reading database ... 12797 files and directories currently installed.)
Preparing to unpack .../python3-minimal_3.7.3-1_amd64.deb ...
Unpacking python3-minimal (3.7.3-1) ...
Selecting previously unselected package libmpdec2:amd64.
Preparing to unpack .../libmpdec2_2.4.2-2_amd64.deb ...
Unpacking libmpdec2:amd64 (2.4.

Setting up apt-transport-https (1.8.2) ...
Setting up distro-info-data (0.41+deb10u1) ...
Setting up binutils-common:amd64 (2.31.1-16) ...
Setting up linux-libc-dev:amd64 (4.19.67-2+deb10u2) ...
Setting up libnpth0:amd64 (1.6-1) ...
Setting up libassuan0:amd64 (2.5.2-1) ...
Setting up libgomp1:amd64 (8.3.0-6) ...
Setting up libasan5:amd64 (8.3.0-6) ...
Setting up make (4.2.1-1.2) ...
Setting up libmpfr6:amd64 (4.0.2-1) ...
Setting up libquadmath0:amd64 (8.3.0-6) ...
Setting up libmpc3:amd64 (1.1.0-1) ...
Setting up libatomic1:amd64 (8.3.0-6) ...
Setting up sudo (1.8.27-1+deb10u1) ...
invoke-rc.d: could not determine current runlevel
invoke-rc.d: policy-rc.d denied execution of start.
Setting up git (1:2.20.1-2+deb10u1) ...
Setting up libmpx2:amd64 (8.3.0-6) ...
Setting up libubsan1:amd64 (8.3.0-6) ...
Setting up libisl19:amd64 (0.20-2) ...
Setting up gpgconf (2.2.12-1+deb10u1) ...
Setting up libcurl4:amd64 (7.64.0-4) ...
Setting up curl (7.64.0-4) ...
Setting up libmpdec2:amd64 (2.4.2-

Removing intermediate container 3f26fa9af87e
 ---> aafee3c7656f
Step 8/11 : ENV CONDA_AUTO_UPDATE_CONDA=false
 ---> Running in 33b4ef06c4e7
Removing intermediate container 33b4ef06c4e7
 ---> 0504e11775f7
Step 9/11 : ENV CONDA_DEFAULT_ENV=$CONDA_ENV_NAME
 ---> Running in 17361c7faac2
Removing intermediate container 17361c7faac2
 ---> 479110dfae7d
Step 10/11 : ENV CONDA_PREFIX=$CONDA_DIR/envs/$CONDA_DEFAULT_ENV
 ---> Running in c4a373598bf2
Removing intermediate container c4a373598bf2
 ---> 8fd2b9a48b73
Step 11/11 : ENV PATH=$CONDA_PREFIX/bin:/opt/conda/bin:$PATH
 ---> Running in 5002b5a13baf
Removing intermediate container 5002b5a13baf
 ---> fe1453bd6a48
Successfully built fe1453bd6a48
Successfully tagged georgedockeraccount/aml-sdk_docker_image:sdk.v1.0.81
Sending build context to Docker daemon  4.608kB
Step 1/17 : FROM continuumio/miniconda3:4.7.12 as AzureMLSDKOnly
 ---> 406f2b43ea59
Step 2/17 : MAINTAINER George Iordanescu <ghiordan@microsoft.com>
 ---> Using cache
 ---> c8b5e5986ff

   900K .......... .......... .......... .......... .......... 11%  136M 0s
   950K .......... .......... .......... .......... .......... 11%  220M 0s
  1000K .......... .......... .......... .......... .......... 12%  150M 0s
  1050K .......... .......... .......... .......... .......... 12%  167M 0s
  1100K .......... .......... .......... .......... .......... 13%  176M 0s
  1150K .......... .......... .......... .......... .......... 14%  131M 0s
  1200K .......... .......... .......... .......... .......... 14%  125M 0s
  1250K .......... .......... .......... .......... .......... 15%  128M 0s
  1300K .......... .......... .......... .......... .......... 15%  177M 0s
  1350K .......... .......... .......... .......... .......... 16%  202M 0s
  1400K .......... .......... .......... .......... .......... 16%  179M 0s
  1450K .......... .......... .......... .......... .......... 17%  159M 0s
  1500K .......... .......... .......... .......... .......... 18%  202M 0s
  1550K ....

Removing intermediate container a98e8769e7c7
 ---> 122dfe34260b
Step 16/17 : RUN rm -rf /tmp/azcopy
 ---> Running in 49b495cb40b8
Removing intermediate container 49b495cb40b8
 ---> 4dfa5630ab6f
Step 17/17 : ENV PATH=/azcopy10/azcopy_linux_amd64_10.3.2:$PATH
 ---> Running in 565ca6168ebb
Removing intermediate container 565ca6168ebb
 ---> 856e6c365612
Successfully built 856e6c365612
Successfully tagged georgedockeraccount/aml-sdk_docker_imageazcli:sdk.v1.0.81
Sending build context to Docker daemon  4.608kB
Step 1/20 : FROM continuumio/miniconda3:4.7.12 as AzureMLSDKOnly
 ---> 406f2b43ea59
Step 2/20 : MAINTAINER George Iordanescu <ghiordan@microsoft.com>
 ---> Using cache
 ---> c8b5e5986ff6
Step 3/20 : RUN apt-get update --fix-missing && apt-get install -y --no-install-recommends     gpg-agent     gcc g++     wget bzip2     curl apt-transport-https lsb-release gpg     git make     sudo &&     apt-get clean &&     rm -rf /var/lib/apt/lists/*
 ---> Using cache
 ---> 9a75836064cf
Step 4/20 :

In [10]:
# import docker

# cli = docker.APIClient()
# response=''
# response = [line for line in cli.build(
#     path=working_path, 
#     dockerfile=docker_file_path, 
#     pull=True,
#     rm=True, 
#     quiet=False,
#     nocache=True,
#     tag=aml_docker_image_name
# )]
# response[100:]

# client = docker.from_env()
# client.containers.list()

In [11]:
local_dir = !pwd
crt_local_dir = os.path.join(*([local_dir.s, 'docker_run_dir02']))
crt_local_dir 
!mkdir -p $crt_local_dir
!rm -rf $crt_local_dir/test_*
!ls -l $crt_local_dir
# ! chmod -R ugo=rwx $crt_local_dir

'/datadrive01/prj/PowerAIWithDocker/amlsdk/docker_run_dir02'

total 8
-rwxrwxrwx 1 root root 5662 Dec  3 03:31 exported_conda_env.yml


In [12]:
# export the resulted conda env

cli_command_inside_container = ' /bin/bash -c "conda env list ; pwd; ls -l /workspace ; ' + \
    'python -c \'import azureml.core;print(azureml.core.VERSION)\'; ' + \
    'conda env export > /workspace/exported_conda_env.yml"'

cli_command='docker run -it --rm  --name aml-sdk_docker_container ' + \
'-v ' + crt_local_dir + ':/workspace:rw ' + \
aml_docker_image_name + \
cli_command_inside_container

cli_command

if 1:
    ! $cli_command
else:
    client = docker.from_env()
    client.containers.run(aml_docker_image_name, 
                          runtime='',
                          remove=True,
                          volumes={'/workspace': {'bind': '/workspace', 'mode': 'rw'}},
                          working_dir='/',
                          command=cli_command_inside_container)

'docker run -it --rm  --name aml-sdk_docker_container -v /datadrive01/prj/PowerAIWithDocker/amlsdk/docker_run_dir02:/workspace:rw georgedockeraccount/aml-sdk_docker_image:sdk.v1.0.81 /bin/bash -c "conda env list ; pwd; ls -l /workspace ; python -c \'import azureml.core;print(azureml.core.VERSION)\'; conda env export > /workspace/exported_conda_env.yml"'

# conda environments:
#
base                     /opt/conda
aml-sdk-conda-env     *  /opt/conda/envs/aml-sdk-conda-env

/
total 8
-rwxrwxrwx 1 root root 5662 Dec  3 03:31 exported_conda_env.yml
1.0.81


In [13]:
saved_conda_dependency_file = 'aml_sdk_conda_dep_file'+os.getenv('image_version')+'.yml'
exported_conda_dependency_file = 'aml_sdk_conda_dep_file_exported'+os.getenv('image_version')+'.yml'

!mkdir -p ./docker_history
!cp $docker_file_path ./docker_history/
!cp $conda_dependency_file_path ./docker_history/$saved_conda_dependency_file
!cp {os.path.join(*([crt_local_dir, 'exported_conda_env.yml']))} ./docker_history/$exported_conda_dependency_file



### To run aml SDK, ssh into a vm, run the command below and then go to: 
your_vm.eastus2.cloudapp.azure.com:9001/  
  
Make sure the host port (9001) is open in the VM.  

In [14]:
!echo docker run -it -p 9001:8888 -v $(pwd)/../:/workspace:rw $aml_docker_image_name /bin/bash -c '"jupyter notebook --notebook-dir=/workspace --ip='*' --port=8888 --no-browser --allow-root"'

docker run -it -p 9001:8888 -v /datadrive01/prj/PowerAIWithDocker/amlsdk/../:/workspace:rw georgedockeraccount/aml-sdk_docker_image:sdk.v1.0.81 /bin/bash -c "jupyter notebook --notebook-dir=/workspace --ip=* --port=8888 --no-browser --allow-root"


### Run short tests
Run ' import azureml.core; print("AML SDK version:", azureml.core.VERSION) ' in plain python and in a notebook.
  
Compare the SDK version against latest AML SDK [relese notes](https://docs.microsoft.com/en-us/azure/machine-learning/service/azure-machine-learning-release-notes)

In [15]:
!echo docker run  $aml_docker_image_name /bin/bash -c "python -c 'import azureml.core;print(azureml.core.VERSION)'"
!docker run  $aml_docker_image_name /bin/bash -c "python -c 'import azureml.core;print(azureml.core.VERSION)'"

!docker run  $aml_azcli_docker_image_name /bin/bash -c "python -c 'import azureml.core;print(azureml.core.VERSION)'; az --version; azcopy -h"
!docker run  $aml_azcli_azdevops_docker_image_name /bin/bash -c "python -c 'import azureml.core;print(azureml.core.VERSION)'; az --version; az devops -h"


docker run georgedockeraccount/aml-sdk_docker_image:sdk.v1.0.81 /bin/bash -c python -c 'import azureml.core;print(azureml.core.VERSION)'
1.0.81
1.0.81
azure-cli                         2.0.78

command-modules-nspkg              2.0.3
core                              2.0.78
nspkg                              3.0.4
telemetry                          1.0.4

Python location '/opt/az/bin/python3'
Extensions directory '/root/.azure/cliextensions'

Python (Linux) 3.6.5 (default, Dec 12 2019, 11:11:10) 
[GCC 6.3.0 20170516]

Legal docs and information: aka.ms/AzureCliLegal



Your CLI is up-to-date.

Please let us know how we are doing: https://aka.ms/clihats
/bin/bash: azcopy: command not found
1.0.81
azure-cli                         2.0.78

command-modules-nspkg              2.0.3
core                              2.0.78
nspkg                              3.0.4
telemetry                          1.0.4

Extensions:
azure-devops                      0.16.0

Python location '/opt/az/bin/pytho

#### Do the same in a  notebook

In [16]:
test_notebook_base_name = 'test_aml_sdk_docker_image'
test_notebook_name = test_notebook_base_name+'.ipynb'

In [17]:
%%writefile $crt_local_dir/$test_notebook_name

{
 "cells": [
  {
   "cell_type": "code",
   "execution_count": 3,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "AML SDK version: 0.1.68\n"
     ]
    }
   ],
   "source": [
    "# Check core SDK version number\n",
    "import azureml.core\n",
    "\n",
    "print(\"AML SDK version:\", azureml.core.VERSION)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": []
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python [conda env:aml-sdk-conda-env]",
   "language": "python",
   "name": "conda-env-aml-sdk-conda-env-py"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.6.6"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 2
}


Writing /datadrive01/prj/PowerAIWithDocker/amlsdk/docker_run_dir02/test_aml_sdk_docker_image.ipynb


In [18]:
!ls -l $crt_local_dir

total 12
-rwxrwxrwx 1 root       root       5690 Dec 25 00:09 exported_conda_env.yml
-rw-rw-r-- 1 loginvm0_1 loginvm0_1  982 Dec 25 00:09 test_aml_sdk_docker_image.ipynb


#### Run the notebook and see the new SDK version


In [19]:
!docker run -v $crt_local_dir:/workspace:rw $aml_docker_image_name /bin/bash -c "jupyter kernelspec list && jupyter nbconvert --ExecutePreprocessor.kernel_name=python3 --execute --to notebook  /workspace/test_aml_sdk_docker_image.ipynb"

Available kernels:
  python3    /opt/conda/envs/aml-sdk-conda-env/share/jupyter/kernels/python3
[NbConvertApp] Converting notebook /workspace/test_aml_sdk_docker_image.ipynb to notebook
[NbConvertApp] Executing notebook with kernel: python3
[NbConvertApp] Writing 982 bytes to /workspace/test_aml_sdk_docker_image.nbconvert.ipynb


In [20]:
output_file = test_notebook_base_name+'.nbconvert.ipynb'
!cat $crt_local_dir/$output_file

{
 "cells": [
  {
   "cell_type": "code",
   "execution_count": 1,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "AML SDK version: 1.0.81\n"
     ]
    }
   ],
   "source": [
    "# Check core SDK version number\n",
    "import azureml.core\n",
    "\n",
    "print(\"AML SDK version:\", azureml.core.VERSION)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": []
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python [conda env:aml-sdk-conda-env]",
   "language": "python",
   "name": "conda-env-aml-sdk-conda-env-py"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.7.5"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 2
}


#### Push image to ([dockerhub](https://hub.docker.com/)) registry (optional step)

In [21]:
# !docker login -u=$docker_login -p=
!docker push $aml_docker_image_name
# !docker push $aml_sudo_docker_image_name
!docker push $aml_azcli_docker_image_name
!docker push $aml_azcli_azdevops_docker_image_name

The push refers to repository [docker.io/georgedockeraccount/aml-sdk_docker_image]

8147ac1f: Preparing 
9a414878: Preparing 
889c5e93: Preparing 
cb249b79: Preparing 
190fd43a: Preparing 


8147ac1f: Pushing  1.743GB/2.934GBPushing  47.23MB/214.4MBPushing    108MB/2.934GB

8147ac1f: Pushed   2.994GB/2.934GBPushing  2.101GB/2.934GBPushing  2.103GB/2.934GBPushing  2.115GB/2.934GBsdk.v1.0.81: digest: sha256:fbd1c29077093c61e7c26452cf231c33f2270673bc75fea20ec9c1263e24dc4b size: 1586
The push refers to repository [docker.io/georgedockeraccount/aml-sdk_docker_imageazcli]

f57006cd: Preparing 
2729ba56: Preparing 
a52ebdbf: Preparing 
8147ac1f: Preparing 
9a414878: Preparing 
889c5e93: Preparing 
cb249b79: Preparing 
190fd43a: Preparing 
a52ebdbf: Pushed   497.5MB/467.8MBraccount/aml-sdk_docker_image sdk.v1.0.81: digest: sha256:9e710344a4533f618ca792bb1bc15b48857c6c8c90cffdf16833803f7386afd6 size: 2218
The push refers to repository [docker.io/georgedockeraccount/aml-sdk_docker_imageazcliazdevops]

860be35f: Preparing 
f57006cd: Preparing 
2729ba56: Preparing 
a52ebdbf: Preparing 
8147ac1f: Preparing 
9a414878: Preparing 
889c5e93: Preparing 
cb249b79: Preparing 
190fd43a: Preparing 
60be35f: Pushed   17.68MB/16.25MBraccount/aml-sdk_docker_imageazcli sdk.v1.0.81

In [22]:
!jupyter nbconvert --to html createAMLSDKDocker.ipynb

[NbConvertApp] Converting notebook createAMLSDKDocker.ipynb to html
[NbConvertApp] Writing 381832 bytes to createAMLSDKDocker.html
